In [96]:
from datetime import datetime 
import pytz 
  
IST = pytz.timezone('Asia/Kolkata') 
print("start time", datetime.now(IST)) 
print("---------------------------------")

start time 2020-10-19 22:57:51.897734+05:30
---------------------------------


In [97]:
import pandas as pd
import csv as csv

In [98]:
listAllMatches=[]

In [99]:
# Parsing drugbank related CSV files and extracting information
dfDB =pd.read_csv('drugbank_drug.tsv',delimiter='\t',encoding='utf-8')
dfDBSalt =pd.read_csv('drugbank_salt.tsv',delimiter='\t',encoding='utf-8')

print(dfDB.columns[2]) #2=name
print("total columns", len(dfDB.columns))

print(dfDB.columns[30])
print(dfDB.columns[31])
print(dfDB.columns[32])
print(dfDB.columns[35])
listNames_DB=[]
listIds_DB=[]
listExactMatchNames_DB_T=[]


listNames_DB=[x.lower() for x in dfDB[dfDB.columns[2].lower()]]
listIds_DB=[x for x in dfDB[dfDB.columns[0]]]

print("total records in column", len(dfDB[dfDB.columns[2]]))                    
print(len(listNames_DB))
print(len(listIds_DB))
print(listIds_DB[0])

#add to dict drrugbank id as value and name as key
dictDBNamesIds=dict(zip(listNames_DB,listIds_DB))

#synonyms
listsynonyms_DB=[]
listsynonyms_DB=[str(x).lower() for x in dfDB[dfDB.columns[30]]]
dictDBSynonymsIds=dict(zip(listsynonyms_DB,listIds_DB))
print("total dictDBSynonymsIds",len(dfDB[dfDB.columns[30]])) 

#brandname
listIntlBrands_DB=[]

listIntlBrands_DB=[str(x).lower() for x in dfDB[dfDB.columns[31]]]
dictDBBrandsIds=dict(zip(listIntlBrands_DB,listIds_DB))
print("total dictDBBrandsIds",len(dfDB[dfDB.columns[31]]))     

listMixturesBrands_DB=[str(x).lower() for x in dfDB[dfDB.columns[32]]]
dictDBMixturesBrandssIds=dict(zip(listMixturesBrands_DB,listIds_DB))
print("total dictDBMixturesBrandssIds",len(dfDB[dfDB.columns[32]]))

listNamesSalt_DB=[x.lower() for x in dfDBSalt[dfDBSalt.columns[1].lower()]]
listIdsSalt_DB=[x for x in dfDBSalt[dfDBSalt.columns[0]]]
dictDBSaltNamesIds=dict(zip(listNamesSalt_DB,listIdsSalt_DB))
print("total dictDBSaltNamesIds",len(dfDBSalt[dfDBSalt.columns[1]]))     

listDescription_DB=[str(x).lower() for x in dfDB[dfDB.columns[35]]]
dictDBDescriptionIds=dict(zip(listDescription_DB,listIds_DB))
print("total dictDBDescriptionIds",len(dfDB[dfDB.columns[35]]))

name
total columns 56
total records in column 13580
13580
13580
DB00001


In [ ]:
print(dfDB.columns)


In [ ]:
# Parsing AACT pass single drug intervention file and extracting information for pass

import pandas as pd
dfFT =pd.read_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/outtrials_pass_rem_dup4_nctid.csv',delimiter=',',encoding='utf-8')

print(dfFT.columns[5])
print("total columns", len(dfFT.columns))
print("columns", dfFT.columns)


In [ ]:
# Mapping logic based on multiple cases

count=0
dataArrFail=dfFT.values
for x in range(len(dfFT.values)):
    print("ss",x)
    break

matchedf=0
nonmatchedf=0

print("len", len(dfFT.values))
print(dfFT.values[0])

header=['nct_id',  'i_id', 'intervention_type',
       'i_name', 'c_id', 'c_name', 'completion_date', 'overall_status',
        'phase', 'label', 'drugbank id']


with open("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/outtrials_nodbid_pass.tsv",'w') as outcsv1: 
    writer1=csv.writer(outcsv1,delimiter='\t',quotechar='|',quoting=csv.QUOTE_MINIMAL,lineterminator='\n')


    #add drugbank id also to csv matches file 
    #dictDBSynonymsIds dictDBBrandsIds dictDBSaltNamesIds
    with open("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/outtrials_dbid_pass.tsv",'w') as outcsv: 
            writer=csv.writer(outcsv,delimiter='\t',quotechar='|',quoting=csv.QUOTE_MINIMAL,lineterminator='\n')
            writer.writerow(header)
            writer1.writerow(header)
            for idd in range(len(dfFT.values)):
                print("inside",idd)               
                match=False
                count=count+1
                drugnameTrials=str(dataArrFail[idd][5]).lower()
                if drugnameTrials in dictDBNamesIds.keys():
                    for name, idval in dictDBNamesIds.items():   
                        if drugnameTrials == name: 
                            matchedf+=1
                            item=dataArrFail[idd]  
                            dbid=idval
                            #print("write", idd)
                            writer.writerow([ item[2],item[3],
                                             item[4], item[5],item[6],item[7],item[8],
                                             item[9],item[10],item[11], dbid])
                            match=True
                            break
                        if match == True:
                            break
                else:
                    #check synonyms:
                    for idx,synonymitem in enumerate(listsynonyms_DB):
                        listAllSynonyms= synonymitem.split('||')                         
                        for intindex, synonym in enumerate(listAllSynonyms):                            
                            if drugnameTrials == synonym: 
                                #print("match on syn",drugnameTrials + synonym)
                                matchedf+=1
                                item=dataArrFail[idd]  
                                dbid=listIds_DB[idx]
                                #print("write", idd)
                                writer.writerow([ item[2],item[3],
                                             item[4], item[5],item[6],item[7],item[8],
                                             item[9],item[10],item[11], dbid])
                                match=True
                                break
                            if match == True:
                                break
                                
                    if match == False: 
                         for idx,itemIntlBrand in enumerate(listIntlBrands_DB):
                            if itemIntlBrand != "nan" or itemIntlBrand != '':
                                listAllBrandNames=[]
                                #chk for || 
                                if '||' in itemIntlBrand:
                                    listAllNames1= itemIntlBrand.split('||')
                                    for Names1 in listAllNames1:
                                        if '::' in Names1:
                                            listAllNames2=Names1.split('::')
                                            for Names2 in listAllNames2:
                                                listAllBrandNames.append(Names2)
                                        else:                             
                                             listAllBrandNames.append(Names1)
                                else:
                                    if '::' in itemIntlBrand:
                                        listAllNames2=itemIntlBrand.split('::')
                                        for Names2 in listAllNames2:
                                            listAllBrandNames.append(Names2)
                                    else:                             
                                        listAllBrandNames.append(itemIntlBrand)
                        
                                for itemBrandName in listAllBrandNames:
                                    
                                    if drugnameTrials in itemBrandName: 
                                        matchedf+=1
                                        #print("match onbrand")
                                        item=dataArrFail[idd]  
                                        dbid=listIds_DB[idx]
                                        #print("write", idd)
                                        writer.writerow([ item[2],item[3],
                                             item[4], item[5],item[6],item[7],item[8],
                                             item[9],item[10],item[11], dbid])
                                        match=True
                                        break
                            if match == True:
                                break
                                
                     
                    if match == False: 
                         for idx,itemIntlBrand in enumerate(listMixturesBrands_DB):
                            if itemIntlBrand != "nan" or itemIntlBrand != '':
                                listAllBrandNames=[]
                                #chk for || 
                                if '||' in itemIntlBrand:
                                    listAllNames1= itemIntlBrand.split('||')
                                    for Names1 in listAllNames1:
                                        if '::' in Names1:
                                            listAllNames2=Names1.split('::')
                                            for Names2 in listAllNames2:
                                                listAllBrandNames.append(Names2)
                                        else:                             
                                             listAllBrandNames.append(Names1)
                                else:
                                    if '::' in itemIntlBrand:
                                        listAllNames2=itemIntlBrand.split('::')
                                        for Names2 in listAllNames2:
                                            listAllBrandNames.append(Names2)
                                    else:                             
                                        listAllBrandNames.append(itemIntlBrand)
                        
                                for itemBrandName in listAllBrandNames:
                                    if drugnameTrials in itemBrandName: 
                                        matchedf+=1
                                        print("match onbrand",drugnameTrials)
                                        item=dataArrFail[idd]  
                                        dbid=listIds_DB[idx]
                                        #print("write", idd)
                                        writer.writerow([ item[2],item[3],
                                             item[4], item[5],item[6],item[7],item[8],
                                             item[9],item[10],item[11], dbid])
                                        match=True
                                        break
                                        
                            if match == True:
                                break
                                
                    if match==False:
                        for idx,itemDesc in enumerate(listDescription_DB):
                            if itemDesc != "nan" or itemDesc != '':
                                listAllDescNames=[]
                                if '||' in itemDesc:
                                    listAllNames1= itemDesc.split('||')
                                    for Names1 in listAllNames1:
                                        if '::' in Names1:
                                            listAllNames2=Names1.split('::')
                                            for Names2 in listAllNames2:
                                                listAllDescNames.append(Names2)
                                        else:                             
                                            listAllDescNames.append(Names1)
                                else:
                                    if '::' in itemDesc:
                                        listAllNames2=itemDesc.split('::')
                                        for Names2 in listAllNames2:
                                            listAllDescNames.append(Names2)
                                    else:                             
                                        listAllDescNames.append(itemDesc)

                                for itemDescName in listAllDescNames:
                                    prefixpart=itemDescName.split(' ')[0]
                                    if drugnameTrials == prefixpart: 
                                        #print("match on desc",drugnameTrials+prefixpart)
                                        matchedf+=1
                                        item=dataArrFail[idd]  
                                        dbid=idval
                                        #print("write", idd)
                                        writer.writerow([ item[2],item[3],
                                             item[4], item[5],item[6],item[7],item[8],
                                             item[9],item[10],item[11], dbid])
                                        match=True
                                        break
                            if match == True:
                                break
                    
                    if match==False:
                        if drugnameTrials in dictDBSaltNamesIds.keys():
                            for name, idval in dictDBSaltNamesIds.items():   
                                if drugnameTrials == name: 
                                    matchedf+=1
                                    #print("match on salt")
                                    item=dataArrFail[idd]  
                                    dbid=idval
                                    #print("write", idd)
                                    writer.writerow([ item[2],item[3],
                                             item[4], item[5],item[6],item[7],item[8],
                                             item[9],item[10],item[11], dbid])
                                    match=True
                                    break
                                
               
                        
                if match==False:
                    nonmatchedf+=1
                    item=dataArrFail[idd]  
                    #print("write no match", idd)
                    writer.writerow([ item[2],item[3],
                                             item[4], item[5],item[6],item[7],item[8],
                                             item[9], item[10],item[11], ''])

print("matchedf=",matchedf)
print("nonmatchedf=",nonmatchedf)

In [ ]:
# Parsing AACT pass single drug intervention file and extracting information for fail

import pandas as pd
dfFT =pd.read_csv('/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/outtrials_fail_rem_dup4_nctid.csv',delimiter=',',encoding='utf-8')

print(dfFT.columns[5])
print("total columns", len(dfFT.columns))
print("columns", dfFT.columns)


In [ ]:
# Mapping logic based on multiple cases for fail

count=0
dataArrFail=dfFT.values

matchedf=0
nonmatchedf=0

print("len", len(dfFT.values))
print(dfFT.values[0])

header=['nct_id',  'i_id', 'intervention_type',
       'i_name', 'c_id', 'c_name', 'completion_date', 'overall_status',
        'phase', 'label', 'drugbank id']

#add drugbank id also to csv matches file 
#dictDBSynonymsIds dictDBBrandsIds dictDBSaltNamesIds
with open("/sas/vidhya/CompoundDb4jV2/ClinicalTrials/MLData/outtrials_dbid_fail.tsv",'w') as outcsv: 
        writer=csv.writer(outcsv,delimiter='\t',quotechar='|',quoting=csv.QUOTE_MINIMAL,lineterminator='\n')
        writer.writerow(header)
        writer1.writerow(header)
        for idd in range(len(dfFT.values)):
            print("inside",idd)               
            match=False
            count=count+1
            drugnameTrials=str(dataArrFail[idd][5]).lower()
            if drugnameTrials in dictDBNamesIds.keys():
                for name, idval in dictDBNamesIds.items():   
                    if drugnameTrials == name: 
                        matchedf+=1
                        item=dataArrFail[idd]  
                        dbid=idval
                        #print("write", idd)
                        writer.writerow([ item[2],item[3],
                                         item[4], item[5],item[6],item[7],item[8],
                                         item[9],item[10],item[11], dbid])
                        match=True
                        break
                    if match == True:
                        break
            else:
                #check synonyms:
                for idx,synonymitem in enumerate(listsynonyms_DB):
                    listAllSynonyms= synonymitem.split('||')                         
                    for intindex, synonym in enumerate(listAllSynonyms):                            
                        if drugnameTrials == synonym: 
                            #print("match on syn",drugnameTrials + synonym)
                            matchedf+=1
                            item=dataArrFail[idd]  
                            dbid=listIds_DB[idx]
                            #print("write", idd)
                            writer.writerow([ item[2],item[3],
                                         item[4], item[5],item[6],item[7],item[8],
                                         item[9],item[10],item[11], dbid])
                            match=True
                            break
                        if match == True:
                            break

                if match == False: 
                     for idx,itemIntlBrand in enumerate(listIntlBrands_DB):
                        if itemIntlBrand != "nan" or itemIntlBrand != '':
                            listAllBrandNames=[]
                            #chk for || 
                            if '||' in itemIntlBrand:
                                listAllNames1= itemIntlBrand.split('||')
                                for Names1 in listAllNames1:
                                    if '::' in Names1:
                                        listAllNames2=Names1.split('::')
                                        for Names2 in listAllNames2:
                                            listAllBrandNames.append(Names2)
                                    else:                             
                                         listAllBrandNames.append(Names1)
                            else:
                                if '::' in itemIntlBrand:
                                    listAllNames2=itemIntlBrand.split('::')
                                    for Names2 in listAllNames2:
                                        listAllBrandNames.append(Names2)
                                else:                             
                                    listAllBrandNames.append(itemIntlBrand)

                            for itemBrandName in listAllBrandNames:

                                if drugnameTrials in itemBrandName: 
                                    matchedf+=1
                                    #print("match onbrand")
                                    item=dataArrFail[idd]  
                                    dbid=listIds_DB[idx]
                                    #print("write", idd)
                                    writer.writerow([ item[2],item[3],
                                         item[4], item[5],item[6],item[7],item[8],
                                         item[9],item[10],item[11], dbid])
                                    match=True
                                    break
                        if match == True:
                            break


                if match == False: 
                     for idx,itemIntlBrand in enumerate(listMixturesBrands_DB):
                        if itemIntlBrand != "nan" or itemIntlBrand != '':
                            listAllBrandNames=[]
                            #chk for || 
                            if '||' in itemIntlBrand:
                                listAllNames1= itemIntlBrand.split('||')
                                for Names1 in listAllNames1:
                                    if '::' in Names1:
                                        listAllNames2=Names1.split('::')
                                        for Names2 in listAllNames2:
                                            listAllBrandNames.append(Names2)
                                    else:                             
                                         listAllBrandNames.append(Names1)
                            else:
                                if '::' in itemIntlBrand:
                                    listAllNames2=itemIntlBrand.split('::')
                                    for Names2 in listAllNames2:
                                        listAllBrandNames.append(Names2)
                                else:                             
                                    listAllBrandNames.append(itemIntlBrand)

                            for itemBrandName in listAllBrandNames:
                                if drugnameTrials in itemBrandName: 
                                    matchedf+=1
                                    print("match onbrand",drugnameTrials)
                                    item=dataArrFail[idd]  
                                    dbid=listIds_DB[idx]
                                    #print("write", idd)
                                    writer.writerow([ item[2],item[3],
                                         item[4], item[5],item[6],item[7],item[8],
                                         item[9],item[10],item[11], dbid])
                                    match=True
                                    break

                        if match == True:
                            break

                if match==False:
                    for idx,itemDesc in enumerate(listDescription_DB):
                        if itemDesc != "nan" or itemDesc != '':
                            listAllDescNames=[]
                            if '||' in itemDesc:
                                listAllNames1= itemDesc.split('||')
                                for Names1 in listAllNames1:
                                    if '::' in Names1:
                                        listAllNames2=Names1.split('::')
                                        for Names2 in listAllNames2:
                                            listAllDescNames.append(Names2)
                                    else:                             
                                        listAllDescNames.append(Names1)
                            else:
                                if '::' in itemDesc:
                                    listAllNames2=itemDesc.split('::')
                                    for Names2 in listAllNames2:
                                        listAllDescNames.append(Names2)
                                else:                             
                                    listAllDescNames.append(itemDesc)

                            for itemDescName in listAllDescNames:
                                prefixpart=itemDescName.split(' ')[0]
                                if drugnameTrials == prefixpart: 
                                    #print("match on desc",drugnameTrials+prefixpart)
                                    matchedf+=1
                                    item=dataArrFail[idd]  
                                    dbid=idval
                                    #print("write", idd)
                                    writer.writerow([ item[2],item[3],
                                         item[4], item[5],item[6],item[7],item[8],
                                         item[9],item[10],item[11], dbid])
                                    match=True
                                    break
                        if match == True:
                            break

                if match==False:
                    if drugnameTrials in dictDBSaltNamesIds.keys():
                        for name, idval in dictDBSaltNamesIds.items():   
                            if drugnameTrials == name: 
                                matchedf+=1
                                #print("match on salt")
                                item=dataArrFail[idd]  
                                dbid=idval
                                #print("write", idd)
                                writer.writerow([ item[2],item[3],
                                         item[4], item[5],item[6],item[7],item[8],
                                         item[9],item[10],item[11], dbid])
                                match=True
                                break



            if match==False:
                nonmatchedf+=1
                item=dataArrFail[idd]  
                #print("write no match", idd)
                writer.writerow([ item[2],item[3],
                                         item[4], item[5],item[6],item[7],item[8],
                                         item[9], item[10],item[11], ''])

print("matchedf=",matchedf)
print("nonmatchedf=",nonmatchedf)

In [132]:
from datetime import datetime 
import pytz 
  
IST = pytz.timezone('Asia/Kolkata') 
print("end time", datetime.now(IST)) 
print("---------------------------------")
  

end time 2020-10-19 23:46:39.297608+05:30
---------------------------------
